

Con esta configuración, el programa debería funcionar correctamente si los archivos necesarios están en la ruta especificada. Verifica que los datos en `maps` sean accesibles y estén completos.

Aquí tienes el código completo sin la variable `VERSION`:



In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion

# === CONFIG ===
DATAROOT = "C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/"
NUM_POINTS = 1024
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === CATEGORÍAS ===
CATEGORIES = [
    'car', 'truck', 'bus', 'trailer', 'pedestrian',
    'motorcycle', 'bicycle', 'construction_vehicle',
    'traffic_cone', 'barrier'
]
CATEGORY2IDX = {cat: i for i, cat in enumerate(CATEGORIES)}

# === DATASET ===
class NuScenesObjectDataset(Dataset):
    def __init__(self, dataroot):
        self.nusc = NuScenes(dataroot=dataroot, verbose=False)
        self.data = []

        for sample in self.nusc.sample:
            lidar_token = sample['data']['LIDAR_TOP']
            lidar_data = self.nusc.get('sample_data', lidar_token)
            lidar_path = os.path.join(dataroot, lidar_data['filename'])
            print(f"Procesando archivo LiDAR: {lidar_path}")
            if not os.path.exists(lidar_path):
                print(f"Archivo LiDAR no encontrado: {lidar_path}")
                continue

            pc = LidarPointCloud.from_file(lidar_path).points[:3, :].T  # (N, 3)
            print(f"Puntos LiDAR cargados: {pc.shape}")

            cs = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])
            pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
            cs_T = transform_matrix(cs['translation'], Quaternion(cs['rotation']), inverse=False)
            pose_T = transform_matrix(pose['translation'], Quaternion(pose['rotation']), inverse=False)
            global_T = pose_T @ cs_T
            points_h = np.hstack((pc, np.ones((pc.shape[0], 1))))  # (N, 4)
            points_global = (global_T @ points_h.T).T[:, :3]

            valid = 0
            skipped = 0

            for ann_token in sample['anns']:
                ann = self.nusc.get('sample_annotation', ann_token)
                category = ann['category_name'].split('.')[0]
                print(f"Procesando categoría: {category}")
                if category not in CATEGORY2IDX:
                    print(f"Categoría no válida: {category}")
                    continue

                center = np.array(ann['translation'])
                size = np.array(ann['size']) / 2
                rot = Quaternion(ann['rotation']).rotation_matrix
                rel = points_global - center
                local = rel @ rot

                mask = np.all(np.abs(local) <= size, axis=1)
                obj_points = points_global[mask]

                if obj_points.shape[0] < 20:
                    skipped += 1
                    continue

                valid += 1

                if obj_points.shape[0] >= NUM_POINTS:
                    idxs = np.random.choice(obj_points.shape[0], NUM_POINTS, replace=False)
                else:
                    idxs = np.random.choice(obj_points.shape[0], NUM_POINTS, replace=True)
                obj_points = obj_points[idxs]
                label = CATEGORY2IDX[category]
                self.data.append((obj_points.astype(np.float32), label))

            print(f"Objetos válidos: {valid}, Skipped: {skipped}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pts, label = self.data[idx]
        return torch.tensor(pts), torch.tensor(label)

# === MODELO POINTNET ===
class PointNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

        self.fc1 = nn.Linear(1024, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.dropout(self.fc2(x))))
        return self.fc3(x)

# === ENTRENAMIENTO ===
def train():
    dataset = NuScenesObjectDataset(DATAROOT)
    print("Número de objetos válidos en el dataset:", len(dataset))

    if len(dataset) == 0:
        print("ERROR: No se encontraron objetos válidos. Revisa la ruta y el contenido del dataset.")
        return

    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

    model = PointNetClassifier(num_classes=NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for pts, labels in loader:
            pts, labels = pts.to(DEVICE), labels.to(DEVICE)
            pts = pts.transpose(1, 2)  # <-- CORRECCIÓN CLAVE
            out = model(pts)
            loss = criterion(out, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

if __name__ == "__main__":
    train()

Procesando archivo LiDAR: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402927647951.pcd.bin
Archivo LiDAR no encontrado: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402927647951.pcd.bin
Procesando archivo LiDAR: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402928147847.pcd.bin
Archivo LiDAR no encontrado: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402928147847.pcd.bin
Procesando archivo LiDAR: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402928698048.pcd.bin
Archivo LiDAR no encontrado: C:/Users/usuario/Desktop/CAF/v1.0-mini_canbus-001/samples/LIDAR_TOP/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402928698048.pcd.bin
Procesando archivo LiDAR:



### Cambios realizados:
1. **Eliminación de la variable `VERSION`**: Ahora el código no depende de esta variable.
2. **Simplificación de la inicialización del dataset**: Se eliminó el parámetro `version` en la clase `NuScenesObjectDataset`.

Este código debería funcionar correctamente si los archivos necesarios están en la ruta especificada. Verifica que los datos en `DATAROOT` sean accesibles y estén completos.